## Gemini 기본 처리

### 라이브러리 설치

In [1]:
%pip install --upgrade --quiet google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 6.7 MB/s eta 0:00:00


### GCP 환경설정 및 로그인

In [2]:
import os

PROJECT_ID = "ai-hangsik" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}
USE_VERTEX_AI = True #@param {type:"boolean"}
MODEL = "gemini-2.5-flash" #@param {type:"string"}

In [ ]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project {PROJECT_ID}

### Vertex AI Client 실행

In [7]:
import base64
from IPython.display import Image, display, Markdown

from google import genai
from google.genai import types
from google.genai.types import HttpOptions, Retrieval, Tool, VertexRagStore

client = genai.Client(
    vertexai=USE_VERTEX_AI,
    project=PROJECT_ID,
    location=REGION,)

In [8]:
RAG_CORPUS_ID = "projects/ai-hangsik/locations/us-central1/ragCorpora/7291609271689543680"

rag_retrieval_tool = Tool(
    retrieval=Retrieval(
        vertex_rag_store=VertexRagStore(
            rag_corpora=[RAG_CORPUS_ID],
            similarity_top_k=10,
            vector_distance_threshold=0.5,
        )
    )
)

### Gemini Functaion calling

In [9]:
from google import genai
from google.genai.types import HttpOptions
from IPython.display import Image, display, Markdown

generate_content_config = types.GenerateContentConfig(
    system_instruction ="""당신을 사용자의 질문에 대해서 RAG 시스템에 있는 정보를 조회해주는 역할을 합니다. """,

    tools = [rag_retrieval_tool]
)

response = client.models.generate_content(
      model = MODEL,
      contents =
        types.Content(
          role="user",
          parts=[
            types.Part.from_text(text="""구글의 2024년 매출은 어떻게 되나요 ?""")
          ]
      ),
      config = generate_content_config,
  )

print(response.text)



2024년 1분기(3월 31일 마감) Alphabet Inc.의 총매출은 805억 3,900만 달러였습니다.


In [10]:
print(response)


sdk_http_response=HttpResponse(
  headers=<dict len=9>
) candidates=[Candidate(
  avg_logprobs=-0.9984837445345792,
  content=Content(
    parts=[
      Part(
        text='2024년 1분기(3월 31일 마감) Alphabet Inc.의 총매출은 805억 3,900만 달러였습니다.'
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  grounding_metadata=GroundingMetadata(
    grounding_chunks=[
      GroundingChunk(
        retrieved_context=GroundingChunkRetrievedContext(
          rag_chunk=RagChunk(
            page_span=RagChunkPageSpan(
              first_page=<... Max depth ...>,
              last_page=<... Max depth ...>
            ),
            text="""Alphabet Announces First Quarter 2024 Results
MOUNTAIN VIEW, Calif. – April 25, 2024 – Alphabet Inc. (NASDAQ: GOOG, GOOGL) today announced financial 
results for the quarter ended March 31, 2024.
Sundar Pichai, CEO, said: “Our results in the first quarter reflect strong performance from Search, YouTube and 
Cloud. We are well under way with o

## End of Document